# Asynchronous Data Import Script Explanation

This notebook explains the script that asynchronously imports data into a database using GraphQL mutations. It uses `aiohttp` for making asynchronous HTTP requests and `async_lru` for caching.

## 1. Introduction

The script is designed to:
1. Authenticate and obtain a token.
2. Perform GraphQL queries and mutations.
3. Handle external ID translation.
4. Import user, group, and membership data from JSON files.
5. Cache results to optimize performance.

## 2. Import Necessary Libraries

The script imports several libraries:
- `json`: To handle JSON data.
- `asyncio`: To handle asynchronous operations.
- `aiohttp`: To make asynchronous HTTP requests.
- `config_check_import` and `config_data` from `utils._config`: To manage configurations.
- `alru_cache`: To cache asynchronous functions.
- `cache` from `functools`: To cache regular functions.


## 3. Define the `DBWriter` Class

The `DBWriter` class handles the following:
1. Obtaining a token for authentication.
2. Performing GraphQL queries and mutations.
3. Translating external IDs.
4. Registering new IDs.

### Explanation of the Code

- `__init__`: Initializes the class with username, password, and token.
- `getToken`: Obtains a token for authentication.
- `queryGQL` and `queryGQL3`: Perform GraphQL queries and mutations.
- `GetQuery`: Retrieves GraphQL queries from files.
- `asyncTranslateID`: Translates external IDs to internal IDs.
- `getAllTypes` and `getTypeId`: Retrieve type information.
- `registerID`: Registers new IDs.
- `Read` and `Create`: Perform read and create operations.


## 4. Define the `execute_mutation` Function

The `execute_mutation` function performs GraphQL mutations concurrently with a limit on the number of concurrent connections.

### Explanation of the Code

- Reads the mutation GraphQL from a file.
- Checks if the data is a list.
- Uses a semaphore to limit concurrent connections.
- Performs the mutation for each item in the data list.
- Writes the results to a file.


In [ ]:
async def execute_mutation(data, db_writer, graphql_dir):
    # Get the mutation graphql from the file
    with open(graphql_dir, "r", encoding="utf-8") as file:
        mutation = file.read()

    # Check if the data is a list
    if not isinstance(data, list):
        raise ValueError("JSON data must be a list!")

    # Limited to 5 concurrent connections at the same time
    semaphore = asyncio.Semaphore(5)

    async def limited_query(mutation, variables):
        async with semaphore:
            return await db_writer.queryGQL(mutation, variables)

    tasks = []

    for item in data:
        if not isinstance(item, dict):
            raise ValueError("Each item must be a dictionary in the JSON data list!")

        variables = item
        tasks.append(limited_query(mutation, variables))

    results = await asyncio.gather(*tasks, return_exceptions=True)
    with open("results.txt", "a", encoding="utf-8") as result_file:
        for result in results:
            if isinstance(result, Exception):
                result_file.write(f"Error: {result}\n")
                print(f"Error: {result}")
            else:
                result_file.write(f"Success: {result}\n")
                print(f"Success: {result}")


## 5. Define Import Functions

The import functions (`import_user`, `import_group`, `import_membership`) use the `execute_mutation` function to perform the import operations for users, groups, and memberships.

### Explanation of the Code

- Calls `execute_mutation` with the appropriate data and GraphQL file.


In [ ]:
@config_check_import("users")
async def import_user(data, db_writer, gql_func):
    await execute_mutation(data, db_writer, gql_func)


@config_check_import("groups")
async def import_group(data, db_writer, gql_func):
    await execute_mutation(data, db_writer, gql_func)


@config_check_import("memberships")
async def import_membership(data, db_writer, gql_func):
    await execute_mutation(data, db_writer, gql_func)


## 6. Define `db_writer_async` Function

The `db_writer_async` function loads the system data from a JSON file and performs the import operations using the `DBWriter` class.

### Explanation of the Code

- Loads system data from `systemdata.json`.
- Initializes the `DBWriter` instance.
- Performs create operations for users and registers external IDs.


In [ ]:
async def db_writer_async():
    with open("systemdata.json", "r", encoding="utf-8") as f:
        systemdata = json.load(f)

    users = systemdata["users"]
    externalids = systemdata["externalids"]

    db_writer = DBWriter()

    for user in users:
        user["id"] = user["id"]
        await db_writer.Create(tableName="users", variables=user)

    for externalid in externalids:
        externalid["id"] = externalid["id"]
        externalid["inner_id"] = externalid["inner_id"]
        externalid["typeid_id"] = externalid["typeid_id"]
        await db_writer.registerID(inner_id=externalid["inner_id"], outer_id=externalid["outer_id"],
                                   type_id=externalid["typeid_id"])


## 7. Define `data_import` Function

The `data_import` function orchestrates the import process by loading data and invoking the appropriate import functions.

### Explanation of the Code

- Loads data from `systemdata.json`.
- Initializes the `DBWriter` instance.
- Runs the import operations for users, groups, and memberships.


In [ ]:
def data_import():
    with open("systemdata.json", "r", encoding="utf-8") as file:
        data = json.load(file)

    loop = asyncio.get_event_loop()
    db_writer = DBWriter()
    user_gql = "gql/user_add.gql"
    group_gql = "gql/group_add.gql"
    membership_gql = "gql/membership_add.gql"

    async def import_data():
        await import_user(data["users"], db_writer, user_gql)
        await import_group(data["groups"], db_writer, group_gql)
        await import_membership(data["memberships"], db_writer, membership_gql)

    loop.run_until_complete(import_data())


## 8. Conclusion

In this notebook, we have broken down the script, explaining each part and its purpose. This script is useful for asynchronously importing data into a database using GraphQL mutations.

Ensure you have the necessary configurations and utility functions available in your project for this script to run correctly.


